In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run -m rpy2.situation

rpy2 version:
3.4.5
Python version:
3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:22:55) 
[GCC 10.3.0]
Looking for R's HOME:
    Environment variable R_HOME: None
    Calling `R RHOME`: /opt/R/mkl/4.0.5/lib/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:
/opt/qt/5.15.2/lib:/opt/R/mkl/4.0.5/lib/R/lib:/opt/openmpi/4.0.5-gcc10.2.0/lib:/opt/intel/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64_lin:/opt/intel/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64_lin:/opt/intel/compilers_and_libraries_2020.4.304/linux/tbb/lib/intel64/gcc4.8:/opt/lapack/lapack-3.10.0-gcc10.2.0-blas:/opt/blas/3.10.0/lib64:/opt/gcc/10.2.0/lib64:/opt/gcc/10.2.0/lib
R version:
    In the PATH: R version 4.0.5 (2021-03-31) -- "Shake and Throw"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/opt/R/mkl/4.0.5/lib/R/include']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', '

In [3]:
import numpy as np
from scipy import optimize as sp_optimize
import logging
import numbers

import os
import tempfile
import subprocess

import rpy2.robjects as robj
import rpy2.robjects.vectors as rvec
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()
from rpy2.robjects.conversion import localconverter

In [4]:
import sys
sys.path.append('/home/saikat/Documents/work/sparse-regression/simulation/eb-linreg-dsc/dsc/functions')
import simulate

def center_and_scale(Z):
    dim = Z.ndim
    if dim == 1:
        Znew = Z / np.std(Z)
        Znew = Znew - np.mean(Znew)
    elif dim == 2:
        Znew = Z / np.std(Z, axis = 0)
        Znew = Znew - np.mean(Znew, axis = 0).reshape(1, -1)
    return Znew

def initialize_ash_prior(k, scale = 2):
    w = np.zeros(k)
    w[0] = 0.0
    w[1:(k-1)] = np.repeat((1 - w[0])/(k-1), (k - 2))
    w[k-1] = 1 - np.sum(w)
    #prior_grid = (np.arange(k) * 5) + 0.1
    #prior_grid *= scale
    sk2 = np.square((np.power(scale, np.arange(k) / k) - 1))
    prior_grid = np.sqrt(sk2)
    return w, prior_grid

n = 200
p = 500
p_causal = 20
pve = 0.7
k = 20

X, y, Xtest, ytest, btrue, strue = simulate.equicorr_predictors(n, p, p_causal, pve, rho = 0.0, seed = 10)
X      = center_and_scale(X)
Xtest  = center_and_scale(Xtest)
wk, sk = initialize_ash_prior(k, scale = 2)

In [5]:
'''
mr.ash.alpha
'''
from mrashpen.inference.mrash_wrapR import MrASHR

mrash_r = MrASHR(option = "rds", debug = True)
mrash_r.fit(X, y, sk, binit = np.zeros(p), winit = wk, s2init = 1.0)

2022-04-06 10:28:46,454 | mrashpen.inference.mrash_wrapR | INFO | Mr.ASH terminated at iteration 1785.



In [6]:
mrashR = importr('mr.ash.alpha')

In [8]:
binit = np.zeros(p)
wkinit = wk.copy()
epstol = 1e-8
convtol = 1e-8
maxiter = 1000
s2init = 1.0
update_pi = True
update_sigma2 = True

n, p = X.shape
r_X      = robj.r.matrix(X, nrow = n, ncol = p)
r_y      = rvec.FloatVector(y)
r_sk2    = rvec.FloatVector(np.square(sk))
r_binit  = rvec.FloatVector(binit)
r_wkinit = rvec.FloatVector(wkinit)
r_tol    = rvec.ListVector({'epstol': epstol,
                        'convtol': convtol})

In [ ]:
r_fit = mrashR.mr_ash(r_X, r_y,
                      standardize = False, intercept = True,
                      max_iter = maxiter,
                      sa2 = r_sk2,
                      beta_init = r_binit,
                      pi = r_wkinit,
                      sigma2 = s2init,
                      update_pi = update_pi,
                      update_sigma2 = update_sigma2,
                      tol = r_tol
                     )